# Imports

In [1]:
import pandas as pd
import numpy as np
import re
import random
from typing import TypeVar, Tuple, Union
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import tensorflow as tf

import optuna

Self = TypeVar("Self")

2024-07-04 17:18:35.744996: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 17:18:35.745099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 17:18:35.873828: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Fonctions utiles

In [2]:
def extract_year_and_title(row):
    """
    Fonction qui extrait l'année d'une chaine de caractères.
    Cela permet de séparer l'année et le titre d'un film
    
    Paramètres:
    ----------
    row: string
        la phrase contenant le titre et l'année de sortie
    
    Retours:
    -------
    Tuple[String, String]
        le titre du film et son année
    """
    match = re.search(r'\((\d{4})\)$', row)
    if match:
        year = match.group(1)
        title = row[:match.start()].strip()
    else:
        year = None
        title = row
    return title, year


def combine_genres(row):
    """
    Fonction pour combiner les genres des deux colonnes en une chaine de caractère (sans les dupliqués)
    
    Paramètres:
    ----------
    row: dataframe pandas
        la ligne du dataframe qui contient les deux colonnes (genre_x et genre_y)
    
    Retours:
    -------
    String
        l'ensemble des genres séparés par des ','
    """
    genrex = row['genres_x']
    genrey = row['genres_y']
    genres = genrex.split('|') + (genrey.split(',') if pd.notna(row['genres_y']) else [])
    #enlever les doublons
    unique_genres = list(set(genres))
    #recréer une string séparée par des ','
    return ','.join(unique_genres)


def split_ratings(
    ratings: np.ndarray, 
    val_size: Union[float, int], 
    test_size: Union[float, int]
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Fonction pour créer des ensembles de validation et de test à partir d'une matrice de notes

    Paramètres:
    ----------
    ratings : np.ndarray
        La matrice des notes (peut être clairsemée, remplie de zéros ou de valeurs manquantes).
    val_size : float ou int
        La taille de l'ensemble de validation.
        Si float, la taille est exprimée en pourcentage de la taille totale du tableau des notes.
        Si int, la taille est exprimée en nombre brut de notes dans l'ensemble de validation.
    test_size : float ou int
        La taille de l'ensemble de test.
        Si float, la taille est exprimée en pourcentage de la taille totale du tableau des notes.
        Si int, la taille est exprimée en nombre brut de notes dans l'ensemble de test.

    Retours:
    -------
    Tuple[np.ndarray, np.ndarray, np.ndarray]
        Le tuple des notes d'entraînement, de validation et de test.
    """
    #Prépare les 3 matrices de résultats
    rows, cols = np.where(ratings > 0)
    val_ratings = np.zeros(ratings.shape)
    test_ratings = np.zeros(ratings.shape)
    train_ratings = ratings.copy()

    total_ratings = len(rows)

    if val_size < 1:  # si c'est un pourcentage
        val_size = int(total_ratings * val_size)
    if test_size < 1:  # si c'est un pourcentage
        test_size = int(total_ratings * test_size)

    total_sample_size = val_size + test_size

    # Récupère aléatoirement les indices pour le set de validation et de test
    sample_indices = random.sample(range(total_ratings), total_sample_size)
    val_indices = sample_indices[:val_size]
    test_indices = sample_indices[val_size:]

    # Remplis le set de validation
    for idx in val_indices:
        row, col = rows[idx], cols[idx]
        #permet de les enlever du set d'entraînement pour les séparer totalement
        train_ratings[row, col] = 0
        val_ratings[row, col] = ratings[row, col]

    # Remplis le set de test
    for idx in test_indices:
        row, col = rows[idx], cols[idx]
        #permet de les enlever du set d'entraînement pour les séparer totalement
        train_ratings[row, col] = 0
        test_ratings[row, col] = ratings[row, col]

    return train_ratings, val_ratings, test_ratings


def normalizeRatings(Y: np.ndarray, R: np.ndarray, axis: int = 1) -> Tuple:
    """
    Prétraiter les données en soustrayant la note moyenne pour chaque film (chaque ligne).
    Inclure uniquement les notes réelles : R(i,j)=1.

    [Ynorm, Ymean] = normalizeRatings(Y, R) normalise Y pour que chaque film
    ait une note de 0 en moyenne. Les films non notés ont alors une note moyenne (0)
    Renvoie la note moyenne dans Ymean.

    Paramètres:
    ----------
    Y : np.ndarray
        Le tableau de notes à normaliser.
    R : np.ndarray
        Le tableau indiquant si un utilisateur a noté un film.
    axis : int
        L'axe numpy à utiliser pour calculer la note moyenne.

    Retours:
    -------
    Tuple
        Le tableau normalisé des notes et la note moyenne.
    """
    Ymean = (np.sum(Y * R, axis=axis) / (np.sum(R, axis=axis) + 1e-12)).reshape(-1, 1)

    if axis == 0:
        Ynorm = Y.T - np.multiply(Ymean, R.T)
    else:
        Ynorm = Y - np.multiply(Ymean, R)

    return (Ynorm, Ymean)


def cofi_cost_func_v(X, W, W_features, b, Y, R,movie_features, lambda_):
    """
    Renvoie le coût pour le filtrage basé sur le contenu
    Vectorisé pour la vitesse. Utilise les opérations TensorFlow pour être compatible avec une boucle d'entraînement personnalisée.

    Paramètres:
    -----------
    X: np.ndarray (num_movies,num_features)
        Matrice des caractéristiques des items
    W: np.ndarray (num_users,num_features)
        Matrice des paramètres des utilisateurs
    W_features: np.ndarray (num_movie_features, num_features)
        Matrice de poids des features des films
    b: np.ndarray (1, num_users)
        Vecteur des paramètres des utilisateurs
    Y: np.ndarray (num_movies,num_users)
        Matrice des notes des utilisateurs pour les films
    R: np.ndarray (num_movies,num_users)
        Matrice où R(i, j) = 1 si le i-ème film a été noté par le j-ème utilisateur
    movie_features: np.ndarray
        Matrice des features des films
    lambda_: float
        Paramètre de régularisation

    Retours:
    -------
    float
        La valeur du coût étant donné les paramètres.
    """
    #Calcule le biais apportés par les caractéristiques (features) des films
    movie_bias = tf.matmul(movie_features, W_features)
    
    # On ajoute ce biais aux caractéristiques déjà connues (X) pour prendre en compte l'influence
    # des features sur les notes des utilisateurs (= ajouts de nouvelles caractéristiques) 
    j = (tf.linalg.matmul(X + movie_bias, tf.transpose(W)) + b - Y) * R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_ / 2) * (
        tf.reduce_sum(X**2) + tf.reduce_sum(W**2) + tf.reduce_sum(W_features**2)
    )
    return J

# Chargement et traitements des données

#### Chargement de movieLens (movies et ratings)

In [3]:
#Movies
df_movies1 = pd.read_csv(
        "./data/movielens_complete/ml-1m/movies.dat",
        sep="::",
        encoding='ISO-8859-1',
        engine='python',
        header=None
    )

df_movies1.columns = ["movieId", "title", "genres"]

#Séparation des titres en titre et années pour pouvoir le lier à l'autre dataset
df_movies1['title'], df_movies1['year'] = zip(*df_movies1['title'].map(extract_year_and_title))


#Ratings
df_ratings1 = pd.read_csv(
        "./data/movielens_complete/ml-1m/ratings.dat",
        sep="::",
        encoding='ISO-8859-1',
        engine='python',
        header=None
    )

df_ratings1.columns = ["userId","movieId","Rating","timestamp"]


#matrice d'interaction utilisateur-films
df_matrix = df_ratings1.pivot(index="movieId", columns="userId", values="Rating")

#### Chargement de IMbd (titleBasics, title_crew et ratings)

In [4]:
#Chargement de titleBasics
df_movies2 = pd.read_csv(
        "./data/IMDb/title.basics.tsv",
        sep='\t'
    )

#Surpression de tout ce qui n'est pas un movie pour pouvoir le combiner plus tard
df_movies2.drop(df_movies2[df_movies2['titleType'] != 'movie'].index, inplace=True)


#Chargement de titleCrew
df_crew = pd.read_csv(
        "./data/IMDb/title.crew.tsv",
        sep='\t'
    )

df_crew.drop(columns=['writers'], inplace=True)

#chargement de ratings
df_ratings2 = pd.read_csv(
        "./data/IMDb/title.ratings.tsv",
        sep='\t'
    )

/tmp/ipykernel_24/3002934010.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies2 = pd.read_csv(


#### Merge des deux datasets

In [5]:
#merge des datasets movies pour obtenir plus de metadata
df_movies = pd.merge(df_movies1, df_movies2, left_on=['title', 'year'], right_on=['primaryTitle', 'startYear'], how='left')
df_movies.drop_duplicates(subset='movieId', keep='first', inplace=True)
 
#merge des movies et de ratings pour avoir les notes moyennes en metadata
df_movies = pd.merge(df_movies, df_ratings2, on='tconst', how='left')

#merge de movies et de crew pour avoir les directors en metadata
df_movies = pd.merge(df_movies, df_crew, on='tconst', how='left')

#nettoyage du df_movies
df_movies['genres'] = df_movies.apply(combine_genres, axis=1)
df_movies['directors'] = df_movies['directors'].fillna('')
df_movies['isAdult'] = df_movies['isAdult'].fillna(0)
#pour ne pas mettre 0 et influencer les choix trop négativement pour ces films, on remplace les nan par la moyenne des averageRating
df_movies['averageRating'] = df_movies['averageRating'].fillna(df_movies['averageRating'].mean())
df_movies['averageRating'] = df_movies['averageRating'] / 2
df_movies.drop(columns=['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'startYear', 'endYear', 'runtimeMinutes', 'genres_x', 'genres_y'], inplace=True)

# Rajout des films qui n'ont jamais été notés pour permettre l'utilisation des features plus tard
movie_ids_in_movies = set(df_movies['movieId'])
movie_ids_in_ratings = set(df_matrix.index)
missing_movie_ids = movie_ids_in_movies - movie_ids_in_ratings
new_entries = pd.DataFrame(np.nan, index=list(missing_movie_ids), columns=df_matrix.columns)
df_matrix = pd.concat([df_matrix, new_entries], ignore_index=False)

/tmp/ipykernel_24/970431456.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_movies['isAdult'] = df_movies['isAdult'].fillna(0)


#### Création des np.array pour le collaborative filtering

In [6]:
R = df_matrix.notna().astype(int).to_numpy()
df_matrix = df_matrix.fillna(0)
Y = df_matrix.to_numpy()

# Features_engineering

- encodage des genres et des directors en matrice binaire pour être utilisé comme features
- utilisation de isAdult et de averageRating (remis sur 5) comme features
- scaling de la matrice de features pour donner le même poids à chacune d'entre elles et pour lui permettre d'être utilisée plus tard dans l'entraînement

Genres et directors sont choisis car les gens ont souvent des préférences pour des genres ou des préférences pour certains directeurs de films.

isAdult est choisi car certaines personnes peuvent préférer des films considérés comme pour les adultes plutôt que des films plus enfantins.

averageRating est choisi car c'est une donnée que les gens peuvent regarder pour choisir un film (si un film est bien noté en moyenne, on aura tendance à le préférer). Cela peut également aider si un film n'est pas beaucoup noté ou n'est pas du tout noté par les utilisateurs.

In [7]:
#transformation des strings en liste pour être utilisé par MultiLabelBinarizer
df_movies['genres'] = df_movies['genres'].apply(lambda x: x.split(','))
df_movies['directors'] = df_movies['directors'].apply(lambda x: x.split(','))

# Transformation en matrice binaire
genres_mlb = MultiLabelBinarizer()
genres_encoded = genres_mlb.fit_transform(df_movies['genres'])
genres_df = pd.DataFrame(genres_encoded, columns=genres_mlb.classes_)

directors_mlb = MultiLabelBinarizer()
directors_encoded = directors_mlb.fit_transform(df_movies['directors'])
directors_df = pd.DataFrame(directors_encoded, columns=directors_mlb.classes_)

features = pd.concat([genres_df,directors_df, df_movies[['isAdult', 'averageRating']]], axis=1)

# Scaling de la matrice de features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

movie_features = tf.constant(features_scaled, dtype=tf.float64)

In [8]:
#  Données utiles
num_movies, num_users = Y.shape
num_movie_features = movie_features.shape[1]
movieList = df_movies["title"].to_list()

#Séparation en train, val et test
Y_train, Y_val, Y_test = split_ratings(Y, val_size=627, test_size=626)
Ynorm, Ymean = normalizeRatings(Y_train, R)

# Modèle de recommendation

In [9]:
class MovieRecommender:

    def __init__(self, num_users, num_movies, num_features, num_movie_features, movieFeatures):
        """
        Initialisation de notre système de recommendation

        Paramètres:
        -----------
        num_user: int
            nombre d'utilisateurs dans notre système
        num_movies: int
            nombre de films
        num_features: int
            nombre de features du système
        num_movie_features: int
            nombre de caractèristiques (features type genre, directors etc) des films
        movieFeatures: tf.tensor
            tensor contenant les caractéristiques (features type genre, directors etc) pour chaque film
        """
        self.num_users = num_users
        self.num_movies = num_movies
        self.num_features = num_features
        self.num_movie_features = num_movie_features
        self.movieFeatures = movieFeatures
        
        tf.random.set_seed(1234)
        
        self.X = tf.Variable(tf.random.normal((self.num_movies, self.num_features), dtype=tf.float64), name="X")
        self.W = tf.Variable(tf.random.normal((self.num_users, self.num_features), dtype=tf.float64), name="W")
        self.b = tf.Variable(tf.random.normal((1, self.num_users), dtype=tf.float64), name="b")
        self.W_features = tf.Variable(tf.random.normal((self.num_movie_features, self.num_features), dtype=tf.float64), name="W_features")
        
        self.mean = None
        
        
    def fit(self, Y, mean, r_mask, iterations, lambda_, learning_r):
        """
        Fonction qui entraîne notre système de recommendation
        
        Paramètres:
        -----------
        Y: np.ndarray
            Le tableau de notes
        mean: np.array
            Le tableau avec les notes moyennes (pour calculer plus tard les résultats)
        r_mask: np.ndarray
            Le tableau indiquant si un utilisateur a noté un film.
        iterations: int
            Le nombre d'itérations à réaliser pour l'entrainement du modèle
        lambda_: float
            Le paramètre de régularisation
        learning_r: float
            Le learning_rate pour l'optimizer
        """
        self.mean = mean
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_r)
        for iter in range(iterations):
            with tf.GradientTape() as tape:
                # Calcule le coût
                cost_value = cofi_cost_func_v(self.X, self.W, self.W_features, self.b, Y, r_mask, self.movieFeatures, lambda_)

            grads = tape.gradient(cost_value, [self.X, self.W, self.W_features, self.b])

            # Effectue une étape de descente de gradient en mettant à jour la valeur des variables pour minimiser la perte.
            optimizer.apply_gradients(zip(grads, [self.X, self.W, self.W_features, self.b]))

            if iter % 20 == 0:
                print(f"Training loss at iteration {iter}: {cost_value:0.1f}")
    
    
    def getResults(self):
        """
        Fonction qui récupère la matrice de scores
        
        Retours:
        -------
        np.ndarray
            Notes des utilisateurs pour chaque film
        """
        movie_bias = tf.matmul(self.movieFeatures, self.W_features)
        p = np.matmul(self.X.numpy() + movie_bias.numpy(), np.transpose(self.W.numpy())) + self.b.numpy()

        # ajout de la moyenne pour obtenir les bonnes valeurs
        pm = p + self.mean
        
        # bloquer les valeurs entre 0 et 5 pour éviter des valeurs trop grandes (possible à cause de l'ajout de la moyenne)
        pm = np.clip(pm, 0, 5)
        
        return pm
    

    def getError(self, Yval, movieList, printExemples=False):
        """
        Fonction qui récupère l'erreur moyenne des prédictions (métrique pour évaluer le modèle)
        
        Paramètres:
        ----------
        Yval: np.ndarray
            Le tableau des notes
        movieList: List
            La liste contenant les titres de tous les films
        printExemples: bool
            Variable qui permet de print certaines prédictions et leur véritable valeur
        
        Retours:
        -------
        float
            l'erreur abolue moyenne de notre modèle
        """
        pm = self.getResults()

        # Récupération des notes de départ
        non_zero_indices = np.nonzero(Yval)
        rows, cols = non_zero_indices
        
        if(printExemples):
            for k in range (5):
                my_ratings = Yval[:,cols[k]]
                my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

                my_predictions = pm[:, k]

                print("\n\nOriginal vs Predicted ratings for",(k + 1), "\n")
                for i in range(len(my_ratings)):
                    if my_ratings[i] > 0:
                        print(
                            f"Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}"
                        )

        predicted_ratings = pm[rows, cols]
        real_ratings = Yval[rows, cols]

        mae = np.mean(np.abs(predicted_ratings - real_ratings))
        
        return mae
        

    def getCoupleRatings(self, user_id1, user_id2):
        """
        Fonction qui permet de récupérer les notes données par un couple de personnes
        
        Paramètres:
        -----------
        user_id1: int
            Le premier utilisateur de notre couple
        user_id2: int
            Le deuxième utilisateur de notre couple
        
        Retours:
        --------
        np.array
            L'array contenant la note donnée par notre couple pour chaque film
        """
        pm = self.getResults()
        
        user_ratings1 = pm[:, user_id1-1]
        user_ratings2 = pm[:, user_id2-1]
        
        # La note d'un couple pour un film correspond à la moyenne de leurs notes respectives
        couple_ratings = (user_ratings1 + user_ratings2) / 2

        return couple_ratings
    

    def getMovieRecommendation(self, user_id1, user_id2, movies, R):
        """
        Fonction qui récupère le meilleur film pour un couple d'utilisateurs
        
        Paramètres:
        -----------
        user_id1: int
            Le premier utilisateur de notre couple
        user_id2: int
            Le deuxième utilisateur de notre couple
        movies: pd.dataframe
            Le dataframe contenant tous les films, leurs caractéristiques etc
        R: np.ndarray
            Le tableau indiquant si un utilisateur a noté un film.
            
        Retours:
        --------
        object:
            object contenant l'id, le titre et le score du film choisi pour ce couple
        """
        couple_ratings = self.getCoupleRatings(user_id1, user_id2)
        
        # Récupère les films vu par chaque utilisateur
        movies_seen1 = R[:, user_id1-1]
        movies_seen2 = R[:, user_id2-1]

        couple_scores = []
        fallback_scores = []

        # Calcule le couple score pour chaque film (la note donnée avec parfois un bonus)
        # Elimine les films vu par les 2 utilisateurs
        for i in range(len(couple_ratings)):
            avg_rating = couple_ratings[i]

            if movies_seen1[i] and movies_seen2[i]:
                # On garde sous la main ceux notés par les deux au cas où il n'y a aucun nouveau film
                fallback_scores.append((i, avg_rating))
                continue
            elif movies_seen1[i] or movies_seen2[i]:
                couple_score = avg_rating
            else:
                # Les films vu par aucun des deux ont un bonus pour les prioritiser
                couple_score = avg_rating + 1

            couple_scores.append((i, couple_score))

        # Choisi le film avec le meilleur couple_score
        if not couple_scores:
            best_movie = max(fallback_scores, key=lambda x: x[1])
        else:
            best_movie = max(couple_scores, key=lambda x: x[1])

        best_movie_id = best_movie[0]
        best_movie_score = best_movie[1]
        best_movie_title = movies.iloc[best_movie_id]['title']

        return {
            'movie_id': best_movie_id + 1,
            'title': best_movie_title,
            'couple_score': best_movie_score
        }
        

# Utilisation du modèle avec Optuna pour trouver les meilleurs hyperparamètres

In [10]:
def objective(trial):
    # Définition des hyperparamètres à optimiser
    # Les valeurs bornes ont été trouvées après plusieurs essais avec optuna
    num_features = trial.suggest_int('num_features', 50, 250)
    lambda_ = trial.suggest_float('lambda', 1e0, 1e1, log=True)
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True)
    iterations = trial.suggest_int('iterations', 200, 500)

    # Création et entraînement du modèle
    # Utilisation de R * (Y_train > 0) pour séparer totalement le train set du reste et ne prendre en compte
    # que les films notés du train set
    recommender = MovieRecommender(num_users, num_movies, num_features, num_movie_features, movie_features)
    recommender.fit(Ynorm, Ymean, R * (Y_train > 0), iterations, lambda_, learning_rate)

    # Évaluation du modèle sur le set de validation
    mae = recommender.getError(Y_val, movieList)
    
    return mae

# Création d'une étude Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)

best_params = study.best_params

# Affichage des meilleurs résultats
print("Meilleurs hyperparamètres trouvés: ", best_params)
print("Meilleure MAE: ", study.best_value)

# Visualisation de l'importance des hyperparamètres
optuna.visualization.plot_param_importances(study)

[I 2024-07-04 17:19:59,173] A new study created in memory with name: no-name-039ecfd0-575b-4449-9c90-9263b792975c


Training loss at iteration 0: 170590508737.1
Training loss at iteration 20: 17007090125.5
Training loss at iteration 40: 3859483561.7
Training loss at iteration 60: 1498740730.9
Training loss at iteration 80: 795819623.3
Training loss at iteration 100: 497459667.1
Training loss at iteration 120: 339847149.2
Training loss at iteration 140: 245966211.3
Training loss at iteration 160: 185777215.7
Training loss at iteration 180: 145113989.0
Training loss at iteration 200: 116497886.5
Training loss at iteration 220: 95679225.1
Training loss at iteration 240: 80107300.3
Training loss at iteration 260: 68182408.1
Training loss at iteration 280: 58863756.9


[I 2024-07-04 17:22:20,132] Trial 0 finished with value: 2.1014375629077726 and parameters: {'num_features': 222, 'lambda': 6.159572328019419, 'learning_rate': 0.0235000042878769, 'iterations': 291}. Best is trial 0 with value: 2.1014375629077726.


Training loss at iteration 0: 167799048826.5
Training loss at iteration 20: 120763488595.9
Training loss at iteration 40: 88561429476.5
Training loss at iteration 60: 66447980874.1
Training loss at iteration 80: 50871436959.6
Training loss at iteration 100: 39598433502.6
Training loss at iteration 120: 31254281107.3
Training loss at iteration 140: 24966304338.2
Training loss at iteration 160: 20158556456.3
Training loss at iteration 180: 16437812836.0
Training loss at iteration 200: 13528076649.2
Training loss at iteration 220: 11231268923.8
Training loss at iteration 240: 9402665057.2
Training loss at iteration 260: 7934993439.1
Training loss at iteration 280: 6747817418.9
Training loss at iteration 300: 5780249477.4
Training loss at iteration 320: 4985832806.9
Training loss at iteration 340: 4328874229.9
Training loss at iteration 360: 3781775420.5
Training loss at iteration 380: 3323067123.7
Training loss at iteration 400: 2935947982.1


[I 2024-07-04 17:25:38,073] Trial 1 finished with value: 2.4545568762392636 and parameters: {'num_features': 219, 'lambda': 7.124984215555025, 'learning_rate': 0.003074012430220078, 'iterations': 409}. Best is trial 0 with value: 2.1014375629077726.


...

Training loss at iteration 0: 66521706284.7
Training loss at iteration 20: 1363220608.3
Training loss at iteration 40: 218080734.4
Training loss at iteration 60: 62346532.4
Training loss at iteration 80: 29817917.3
Training loss at iteration 100: 18908354.6
Training loss at iteration 120: 13531083.6
Training loss at iteration 140: 10286620.7
Training loss at iteration 160: 8137267.5
Training loss at iteration 180: 6636754.8
Training loss at iteration 200: 5550187.0
Training loss at iteration 220: 4740374.2
Training loss at iteration 240: 4122214.2
Training loss at iteration 260: 3640578.2
Training loss at iteration 280: 3258595.3
Training loss at iteration 300: 2950905.1
Training loss at iteration 320: 2699595.8
Training loss at iteration 340: 2491778.0
Training loss at iteration 360: 2317994.4
Training loss at iteration 380: 2171189.9
Training loss at iteration 400: 2046039.4
Training loss at iteration 420: 1938390.6


[I 2024-07-04 18:36:17,704] Trial 23 finished with value: 1.3379463482379748 and parameters: {'num_features': 87, 'lambda': 2.4764707697674186, 'learning_rate': 0.06503013217197169, 'iterations': 434}. Best is trial 12 with value: 0.9384011778903282.


Training loss at iteration 0: 84491121514.1
Training loss at iteration 20: 5940363784.0
Training loss at iteration 40: 1242400195.0
Training loss at iteration 60: 467642140.8
Training loss at iteration 80: 246350897.8
Training loss at iteration 100: 153176480.9
Training loss at iteration 120: 103732089.5
Training loss at iteration 140: 74188262.5
Training loss at iteration 160: 55264783.4
Training loss at iteration 180: 42529420.0
Training loss at iteration 200: 33616193.8
Training loss at iteration 220: 27172439.4
Training loss at iteration 240: 22384716.3
Training loss at iteration 260: 18743287.9
Training loss at iteration 280: 15917332.1
Training loss at iteration 300: 13685504.5
Training loss at iteration 320: 11895692.6
Training loss at iteration 340: 10440862.3
Training loss at iteration 360: 9244112.3
Training loss at iteration 380: 8249056.5
Training loss at iteration 400: 7413778.2
Training loss at iteration 420: 6706546.2
Training loss at iteration 440: 6103023.0
Training lo

[I 2024-07-04 18:40:04,875] Trial 24 finished with value: 1.647171514056583 and parameters: {'num_features': 110, 'lambda': 1.5682243496989827, 'learning_rate': 0.03354708660953918, 'iterations': 469}. Best is trial 12 with value: 0.9384011778903282.


Meilleurs hyperparamètres trouvés:  {'num_features': 55, 'lambda': 1.7920209899189683, 'learning_rate': 0.09001293307956092, 'iterations': 498}
Meilleure MAE:  0.9384011778903282


# Résultats

In [11]:
# Utilisation des meilleurs paramètres pour entrainer notre modèle
recommender_final = MovieRecommender(num_users, num_movies, best_params['num_features'], num_movie_features, movie_features)
recommender_final.fit(Ynorm, Ymean, R * (Y_train >0), best_params['iterations'], best_params['lambda'], best_params['learning_rate'])

# Obtention de l'erreur moyenne de notre modèle sur le validation set et le test set pour visualiser ses performances
mae_val = recommender_final.getError(Y_val, movieList)
mae_test = recommender_final.getError(Y_test, movieList)

print(f"\n\nErreur absolue moyenne sur val set(MAE) : {mae_val}")
print(f"\n\nErreur absolue moyenne sur test set(MAE) : {mae_test}")

Training loss at iteration 0: 42240203872.7
Training loss at iteration 20: 545007061.8
Training loss at iteration 40: 78530243.5
Training loss at iteration 60: 18062168.1
Training loss at iteration 80: 7443964.4
Training loss at iteration 100: 4520481.4
Training loss at iteration 120: 3267785.4
Training loss at iteration 140: 2561202.9
Training loss at iteration 160: 2107518.3
Training loss at iteration 180: 1796010.4
Training loss at iteration 200: 1572563.7
Training loss at iteration 220: 1406865.1
Training loss at iteration 240: 1280603.3
Training loss at iteration 260: 1182150.1
Training loss at iteration 280: 1103835.3
Training loss at iteration 300: 1040439.6
Training loss at iteration 320: 988316.1
Training loss at iteration 340: 944858.1
Training loss at iteration 360: 908164.8
Training loss at iteration 380: 876825.6
Training loss at iteration 400: 849776.6
Training loss at iteration 420: 826204.6
Training loss at iteration 440: 805479.4
Training loss at iteration 460: 787107.

In [12]:
#Recommendation pour les utilisateurs 1 et 2
best_movie = recommender_final.getMovieRecommendation(1,2, df_movies, R)
best_movie

{'movie_id': 9, 'title': 'Sudden Death', 'couple_score': 6.0}